# Sales Office

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
import pathlib
from plotting import Prices, shades

Load price history

In [ ]:
prices = Prices()

In [ ]:
aerospace = ['SEP','Luxury Jet','Jumbo','SOR','BFR','Satellite']

In [ ]:
mapper = {'Sin': 'SEP', 'Lux':'Luxury Jet', 'Jum': 'Jumbo', 'Sub': 'SOR', 'Sat': 'Satellite'}

dfs = []
for file in pathlib.Path('SalesOfficePrices').glob('*.hdf'):
    try:
        df = pd.read_hdf(file, 'prices').rename(columns=mapper)
    except:
        df = pd.read_hdf(file, 'test')
    dfs.append(df)
    
df0 = pd.DataFrame(columns=['Time', 'Economy', 'SEP', 'Luxury Jet', 'Jumbo', 'SOR', 'BFR', 'Satellite'])
as_prices = pd.concat([df0] + dfs, join='outer').sort_values('Time')
as_prices

Date of the nerf!

In [ ]:
nerf = np.datetime64('2020-11-26T00')

Plot the history of sales office prices

In [ ]:
f, axs = plt.subplots(3, 2, figsize=(14,14))
axr = axs.ravel()
for commodity, ax in zip(aerospace, axr):
    sns.scatterplot(x='Time', y=commodity, label='Prices', data=as_prices, ax=ax)
    ax.set_ylabel('Price ($)')
    ax.set_title(commodity)
    for tick in ax.get_xticklabels():
        tick.set_rotation(45)
    ylims = ax.get_ylim()    

    economy = prices.economy_history
    es, indices = np.unique([e[1] for e in prices.economy_history], return_index=True)
    ylim = ax.get_ylim()
    for i in range(len(economy) - 1):
        label = economy[i][1] if i in indices else None
        ax.fill_between([economy[i][0], economy[i+1][0]], ylim[0]*np.ones(2), 
            (0.03*ylim[1]+0.97*ylim[0])*np.ones(2), color=shades[economy[i][1]], label=label
        )

    ax.plot([nerf,nerf], ylims, 'k--', label='Nerf')
    ax.legend(loc='upper right')

plt.tight_layout()
sns.despine()

I changed the economy strings from 'N' to 'N0' for normal economy before the nerf. This allows us to assess how the prices have changed since.

In [ ]:
economy_labels = {'N0': 'Normal (pre-nerf)', 'N': 'Normal', 'R': 'Recession', 'B': 'Boom'}
as_prices['Economy'] = as_prices['Economy'].apply(lambda k: economy_labels[k])

Plot the distribution of prices by economy

In [ ]:
f, axs = plt.subplots(2, 3, figsize=(14,7))
axr = axs.ravel()
for commodity, ax in zip(aerospace, axr):
    try:
        sns.histplot(data=as_prices, x=commodity, hue='Economy', ax=ax, kde=True, kde_kws={'bw_method':2})
    except:
        sns.histplot(data=as_prices, x=commodity, hue='Economy', ax=ax)
    ax.xaxis.set_major_formatter(ticker.EngFormatter())

sns.despine()

#f.savefig('sales_office.pdf')

## Bonus
What is the distribution of quality bonus values?

In [ ]:
as_prices['Bonus'].hist(bins=15, xrot=45, grid=False)
sns.despine()

It looks like a uniform distribution between 1 and 3!

## Quantities

In [ ]:
mapper = {'Sin': 'SEP', 'Lux':'Luxury Jet', 'Jum': 'Jumbo', 'Sub': 'SOR', 'Sat': 'Satellite'}

dfs = []
for file in pathlib.Path('SalesOfficeQuantities').glob('*.hdf'):
    try:
        df = pd.read_hdf(file, 'quantities').rename(columns=mapper)
    except:
        pass
    dfs.append(df)
    
as_quantities = pd.concat([df0] + dfs, join='outer').sort_values('Time').fillna(0)
as_quantities.head()

Plot the distribution of quantities for each sales office product 

In [ ]:
f, axs = plt.subplots(2, 3, figsize=(14,6))
axr = axs.ravel()
for commodity, ax in zip(aerospace, axr):
    positives = as_quantities[commodity] > 0
    sns.histplot(data = as_quantities[positives], x=commodity, ax=ax, bins=np.linspace(-0.5, 5.5, 7))
plt.tight_layout()
sns.despine()

**Conclusions.**
It seems that the quantity of each commodity is uniformly-distributed, but across slightly different supports. For SEP, it's uniform over {2, 3, 4, 5}, while for the others, it seems to be uniformly-distributed across {1, 2, 3, 4}.

## Frequencies
Use this to figure out how many of each commodity is needed per day, by assuming that the demand is equal to the average quantity of each product multipled by one-half of the number of Sales Office levels you have.

In [ ]:
p = np.array([3.5, 2.5, 2.5, 2.5, 2.5, 2.5])
n = as_quantities.shape[0]
freq = [len(np.where(as_quantities[commodity]>0)[0])/n for commodity in aerospace]
plt.bar(x=aerospace, height=freq)
sns.despine()

levels_day = 65 / 2
df = pd.DataFrame()
df['Commodity'] = aerospace
df['Expected /d'] = freq * p * levels_day
df

## Stats
Approximate the mean prices for each commodity, by economy

In [ ]:
def get_stats(commodity, economy):
    x = as_prices[as_prices['Economy'] == economy][commodity]
    return [x.mean(), x.std()]

recession = [get_stats(commodity, 'Recession') for commodity in aerospace]
normal = [get_stats(commodity, 'Normal') for commodity in aerospace]
boom = [get_stats(commodity, 'Boom') for commodity in aerospace]

df = pd.DataFrame(np.hstack([recession, normal, boom]), columns=['Recession Mean','Recession Std. dev.','Normal Mean','Normal Std. dev.','Boom Mean','Boom Std. dev.'], index=aerospace)
df

What's the increase in price in Boom?

In [ ]:
df['Boom Mean'] / df['Normal Mean']

What's the decrease in price in Recession?

In [ ]:
df['Normal Mean'] / df['Recession Mean']